In [25]:
import mlflow
mlflow.set_experiment("Sentiment Analysis Using MLOPS ")

2024/03/28 21:08:23 INFO mlflow.tracking.fluent: Experiment with name 'Sentiment Analysis Using MLOPS ' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/Hello/Desktop/MLOPS/mlruns/609289338975354017', creation_time=1711640303226, experiment_id='609289338975354017', last_update_time=1711640303226, lifecycle_stage='active', name='Sentiment Analysis Using MLOPS ', tags={}>

In [26]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [27]:
df = pd.read_csv(r'C:\Users\Hello\INTERNSHIP\reviews_badminton\data.csv')

In [28]:
df

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1
...,...,...,...,...,...,...,...,...
8513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
8514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
8515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
8516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8518 entries, 0 to 8517
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Reviewer Name    8508 non-null   object 
 1   Review Title     8508 non-null   object 
 2   Place of Review  8468 non-null   object 
 3   Up Votes         8508 non-null   float64
 4   Down Votes       8508 non-null   float64
 5   Month            8053 non-null   object 
 6   Review text      8510 non-null   object 
 7   Ratings          8518 non-null   int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 532.5+ KB


In [30]:
df_2= df[['Ratings','Review text']]

In [31]:
df_2 = df_2.dropna()
df_2.isnull().sum()

Ratings        0
Review text    0
dtype: int64

In [32]:
df_2['Ratings'] = df_2['Ratings'].apply(lambda x: 'Negative' if x < 3 else ('Neutral' if x == 3 else 'Positive'))


In [33]:
df_2

,Ratings,Review text
0,Positive,"Nice product, good quality, but price is now r..."
1,Negative,They didn't supplied Yonex Mavis 350. Outside ...
2,Negative,Worst product. Damaged shuttlecocks packed in ...
3,Neutral,"Quite O. K. , but nowadays the quality of the..."
4,Negative,Over pricedJust â?¹620 ..from retailer.I didn'...
...,...,...
8505,Neutral,Delivered before time but price is high from m...
8506,Positive,up to the mark but same is available in market...
8507,Positive,Nice delivery speedREAD MORE
8508,Positive,No complaints about the item . Its the best on...


In [34]:
# Step 3: Data Preprocessing on train data (X_train)

import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize WordNet lemmatizer
lemmatizer = WordNetLemmatizer()


In [35]:
def clean(doc): # doc is a string of text
    doc = doc.replace("READ MORE", "")
    
    # Remove punctuation and numbers.
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])

    # Converting to lower case
    doc = doc.lower()
    
    # Tokenization
    tokens = nltk.word_tokenize(doc)

    # Lemmatize
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Stop word removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in lemmatized_tokens if word.lower() not in stop_words]
    
    # Join and return
    return " ".join(filtered_tokens)

In [36]:
y = df_2.pop('Ratings')
X=df_2
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=45)
X_train.shape,y_test.shape

((5957, 1), (2553,))

In [37]:
# Improving the efficiency by applying cleaning the text data before hand
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()
%time X_train_clean = X_train['Review text'].progress_apply(lambda text: clean(text))

X_train_clean.shape

100%|████████████████████████████████████████████████████████████████████████████| 5957/5957 [00:03<00:00, 1690.67it/s]

CPU times: total: 3.44 s
Wall time: 3.53 s


(5957,)

In [38]:
%time X_test_clean = X_test['Review text'].progress_apply(lambda text: clean(text))

X_test_clean.shape

100%|████████████████████████████████████████████████████████████████████████████| 2553/2553 [00:01<00:00, 1632.22it/s]

CPU times: total: 1.48 s
Wall time: 1.57 s


(2553,)

In [39]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# Assuming X_train and y_train are your training data
# Assuming X_test and y_test are your test data

# Define pipelines with the best parameters
pipelines = {
    'DecisionTree': Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', DecisionTreeClassifier(max_depth=10, min_samples_split=2))
    ]),
    'Logistic': Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression(C=1, penalty='l2'))
    ]),
    'SVM': Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', SVC(C=1, kernel='rbf'))
    ]),
    # Add pipelines for other algorithms with their best parameters
}

# Fit the pipelines with training data
for algo_name, pipeline in pipelines.items():
    pipeline.fit(X_train_clean, y_train)

# Evaluate model on test data
for algo_name, pipeline in pipelines.items():
    y_pred = pipeline.predict(X_test_clean)
    accuracy = f1_score(y_test, y_pred,average='micro')
    print(f"Accuracy for {algo_name}: {accuracy}")


Accuracy for DecisionTree: 0.8515471993732864
Accuracy for Logistic: 0.8629063846455151
Accuracy for SVM: 0.8609479044261653


In [40]:
new_data = [
    "Good to ","Bad to buy","avg product"
]

new_data_clean = [clean(doc) for doc in new_data]

prediction = pipeline.predict(new_data_clean)

print("Prediction:", prediction)

Prediction: ['Positive' 'Negative' 'Neutral']


In [43]:
import warnings
warnings.filterwarnings('ignore')


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
import joblib
import time
from joblib import Memory
import os
import mlflow
# Define a memory object to cache intermediate results
cachedir = '.cache'
memory = Memory(location=cachedir, verbose=0)

pipelines = {
    'naive_bayes': Pipeline([
        ('vectorization', CountVectorizer()),
        ('classifier', MultinomialNB())
    ], memory=memory),
    
    'random_forest': Pipeline([
        ('vectorization', CountVectorizer()),
        ('classifier', RandomForestClassifier())
    ], memory=memory),
    
    'logistic_regression': Pipeline([
        ('vectorization', CountVectorizer()),
        ('classifier', LogisticRegression())
    ], memory=memory),
    
    'svm': Pipeline([
        ('vectorization', CountVectorizer()),
        ('classifier', SVC())
    ], memory=memory),

    'knn': Pipeline([
        ('vectorization', CountVectorizer()),
        ('classifier', KNeighborsClassifier())
    ], memory=memory)
}

# Define parameter grid for each algorithm
param_grids = {
    'naive_bayes': [
        {
            'vectorization': [CountVectorizer()],
            'vectorization__max_features' : [1000, 1500, 2000], 
            'classifier__alpha' : [0.1, 0.5, 1, 10]
        }
    ],
    'random_forest': [
        {
            'vectorization': [CountVectorizer()],
            'vectorization__max_features' : [1000, 1500, 2000],
            'classifier__max_depth': [None, 5, 10],
            'classifier__n_estimators': [100, 200, 300]
        }
    ],
    'logistic_regression': [
        {
            'vectorization': [CountVectorizer(), TfidfVectorizer()],
            'vectorization__max_features' : [1000, 2000], 
            'classifier__C': [0.1, 1, 10], 
            'classifier__penalty': ['l1', 'l2'], 
            'classifier__solver': ['liblinear', 'saga'],
        }
    ],
    'knn': [
        {
            'vectorization': [CountVectorizer()],
            'vectorization__max_features' : [1000, 1500, 2000], 
            'classifier__n_neighbors': [3, 5, 7],
            'classifier__p': [1, 2]
        }
    ],
    'svm': [
        {
            'vectorization': [CountVectorizer()],
            'vectorization__max_features' : [1000, 1500, 2000], 
            'classifier__C': [0.1, 1, 10],
            'classifier__kernel': ['linear', 'rbf'],
            'classifier__gamma': ['scale', 'auto']
        }
    ],
}
mlflow.set_experiment('flipkart-sentiment-analysis')

2024/03/28 21:28:51 INFO mlflow.tracking.fluent: Experiment with name 'flipkart-sentiment-analysis' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/Hello/Desktop/MLOPS/mlruns/757650632124654277', creation_time=1711641531828, experiment_id='757650632124654277', last_update_time=1711641531828, lifecycle_stage='active', name='flipkart-sentiment-analysis', tags={}>

In [45]:
# Perform GridSearchCV for each algorithm and log in the mlflow
dev = 'Akula Nikhil'
best_models = {}


for algo in pipelines.keys():
    print("*"*10, algo, "*"*10)
    clf = GridSearchCV(estimator=pipelines[algo], 
                               param_grid=param_grids[algo], 
                               cv=3, 
                               scoring='accuracy', 
                               return_train_score=True,
                               verbose=1
                              )

    #autologging
    mlflow.sklearn.autolog(max_tuning_runs=None)
    
    with mlflow.start_run() as run:
        %time clf.fit(X_train_clean, y_train)
        
    print('Train Score: ', clf.best_score_)
    print('Test Score: ', clf.score(X_test_clean, y_test))

    best_models[algo] = clf.best_estimator_
    print()


********** naive_bayes **********


2024/03/28 21:30:52 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 3 folds for each of 12 candidates, totalling 36 fits
CPU times: total: 14.8 s
Wall time: 28.3 s
Train Score:  0.8623459937446255
Test Score:  0.864081472777125

********** random_forest **********


2024/03/28 21:31:20 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 3 folds for each of 27 candidates, totalling 81 fits
CPU times: total: 7min 10s
Wall time: 7min 34s
Train Score:  0.8541202354347773
Test Score:  0.854289071680376

********** logistic_regression **********


...

Fitting 3 folds for each of 48 candidates, totalling 144 fits
CPU times: total: 1min 41s
Wall time: 1min 55s
Train Score:  0.8616749657341103
Test Score:  0.8597728162945554

********** svm **********


2024/03/28 21:40:52 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 3 folds for each of 36 candidates, totalling 108 fits
CPU times: total: 2min 48s
Wall time: 3min 5s
Train Score:  0.8568058694657735
Test Score:  0.8558558558558559

********** knn **********


2024/03/28 21:43:59 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.


Fitting 3 folds for each of 18 candidates, totalling 54 fits
CPU times: total: 2min 9s
Wall time: 1min 22s
Train Score:  0.8499237737208317
Test Score:  0.8511555033294164

